In [8]:
!pip install keras==2.7.0
!pip install tensorflow==2.5.0

# Image Augmentation

In [9]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [10]:
train_datagen=ImageDataGenerator(rescale=1./255,zoom_range=0.2,horizontal_flip=True,vertical_flip=False)

In [11]:
test_datagen=ImageDataGenerator(rescale=1./255)

In [12]:
ls

Dataset/


In [13]:
pwd

'/home/wsuser/work'

In [14]:

import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bHfE0z3tYuwrhV-z0neNxGqprEFMVR3oO8JFhqdxMLD0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'forestfiredetection-donotdelete-pr-kwf7h4pt0wk0fy'
object_key = 'Forest fire.zip'

streaming_body_6 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/


In [15]:
from io import BytesIO
import zipfile
unzip = zipfile.ZipFile(BytesIO(streaming_body_6.read()), "r")
file_paths = unzip.namelist() 
for path in file_paths:
    unzip.extract(path)

In [16]:
pwd

'/home/wsuser/work'

In [17]:
import os
filenames = os.listdir('/home/wsuser/work/Forest fire/forest_fire/Training and Validation')

In [18]:
x_train=train_datagen.flow_from_directory("/home/wsuser/work/Forest fire/forest_fire/Training and Validation",target_size=(128,128),class_mode='categorical',batch_size=24)

Found 1832 images belonging to 2 classes.


In [19]:
x_test=test_datagen.flow_from_directory(r"/home/wsuser/work/Forest fire/forest_fire/Testing",target_size=(128,128),
                                        class_mode='categorical',batch_size=24)

Found 68 images belonging to 2 classes.


In [20]:
x_train.class_indices

{'fire': 0, 'nofire': 1}

# CNN

In [21]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense,Convolution2D,MaxPooling2D,Flatten

In [22]:
model=Sequential()

2022-11-21 07:31:32.758191: W tensorflow/stream_executor/platform/default/dso_loader.cc:64] Could not load dynamic library 'libcuda.so.1'; dlerror: libcuda.so.1: cannot open shared object file: No such file or directory; LD_LIBRARY_PATH: /opt/ibm/dsdriver/lib:/opt/oracle/lib:/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow
2022-11-21 07:31:32.758250: W tensorflow/stream_executor/cuda/cuda_driver.cc:326] failed call to cuInit: UNKNOWN ERROR (303)


In [23]:
model.add(Convolution2D(32,(3,3),input_shape=(128,128,3),activation='relu'))

In [24]:
model.add(MaxPooling2D(pool_size=(2,2)))

In [25]:
model.add(Flatten())

In [26]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d (Conv2D)              (None, 126, 126, 32)      896       
_________________________________________________________________
max_pooling2d (MaxPooling2D) (None, 63, 63, 32)        0         
_________________________________________________________________
flatten (Flatten)            (None, 127008)            0         
Total params: 896
Trainable params: 896
Non-trainable params: 0
_________________________________________________________________


In [27]:
32*(3*3*3+1)

896

# Hidden Layers

In [28]:
model.add(Dense(300,activation='relu'))
model.add(Dense(150,activation='relu'))

# Output Layer

In [29]:
model.add(Dense(2,activation='softmax'))

In [30]:
model.compile(loss='categorical_crossentropy',optimizer='adam',metrics=['accuracy'])

In [31]:
len(x_train)

77

In [32]:
1238/24

51.583333333333336

In [33]:
model.fit_generator(x_train,steps_per_epoch=len(x_train),validation_data=x_test,validation_steps=len(x_test),epochs=10)

/opt/conda/envs/Python-3.9/lib/python3.9/site-packages/tensorflow/python/keras/engine/training.py:1940: UserWarning: `Model.fit_generator` is deprecated and will be removed in a future version. Please use `Model.fit`, which supports generators.
  warnings.warn('`Model.fit_generator` is deprecated and '


Epoch 1/10
77/77 [==============================] - 58s 741ms/step - loss: 0.9698 - accuracy: 0.8657 - val_loss: 0.5529 - val_accuracy: 0.8971
Epoch 2/10
77/77 [==============================] - 55s 717ms/step - loss: 0.1668 - accuracy: 0.9383 - val_loss: 0.3615 - val_accuracy: 0.8824
Epoch 3/10
77/77 [==============================] - 56s 722ms/step - loss: 0.1439 - accuracy: 0.9520 - val_loss: 0.3120 - val_accuracy: 0.8971
Epoch 4/10
77/77 [==============================] - 57s 736ms/step - loss: 0.1347 - accuracy: 0.9509 - val_loss: 0.3402 - val_accuracy: 0.9118
Epoch 5/10
77/77 [==============================] - 58s 755ms/step - loss: 0.1265 - accuracy: 0.9552 - val_loss: 0.3401 - val_accuracy: 0.9412
Epoch 6/10
77/77 [==============================] - 54s 700ms/step - loss: 0.1274 - accuracy: 0.9552 - val_loss: 0.3368 - val_accuracy: 0.8971
Epoch 7/10
77/77 [==============================] - 53s 686ms/step - loss: 0.0914 - accuracy: 0.9678 - val_loss: 0.4035 - val_accuracy: 0.9118

# Saving Model

In [41]:
ls

 Dataset/   forest13.h5  'Forest fire'/   modell.h5   Train-model_new.tgz


In [38]:
model.save('forest13.h5')

In [40]:
!tar -zcvf Train-model_new.tgz forest13.h5

forest13.h5


In [42]:
ls -1

Dataset/
forest13.h5
'Forest fire'/
modell.h5
Train-model_new.tgz


# IBM Cloud Deployment Model 

In [43]:
!pip install watson-machine-learning-client --upgrade

     |████████████████████████████████| 538 kB 9.0 MB/s eta 0:00:01


In [47]:
from ibm_watson_machine_learning import APIClient

wml_credentials = {
                    "url": "https://us-south.ml.cloud.ibm.com",
                   "apikey":"Cr7tVxz63aEng9jQaz_ynRfJ-x87lBxhokrZDESUG4SO" 
                  }

client = APIClient(wml_credentials)

In [48]:
client = APIClient(wml_credentials)

In [49]:
def guid_from_space_name(client, space_name): 
    space = client.spaces.get_details()
    return(next(item for item in space['resources'] if item['entity']["name"]==space_name)['metadata']['id'])

In [60]:
space_uid = guid_from_space_name(client, 'model') 
print("Space UID = " + space_uid)

Space UID = c010e556-1f97-4d5d-b8c0-03a3d92f06d3


In [61]:
client.set.default_space(space_uid)

'SUCCESS'

In [62]:
client.software_specifications.list()

-----------------------------  ------------------------------------  ----
NAME                           ASSET_ID                              TYPE
default_py3.6                  0062b8c9-8b7d-44a0-a9b9-46c416adcbd9  base
kernel-spark3.2-scala2.12      020d69ce-7ac1-5e68-ac1a-31189867356a  base
pytorch-onnx_1.3-py3.7-edt     069ea134-3346-5748-b513-49120e15d288  base
scikit-learn_0.20-py3.6        09c5a1d0-9c1e-4473-a344-eb7b665ff687  base
spark-mllib_3.0-scala_2.12     09f4cff0-90a7-5899-b9ed-1ef348aebdee  base
pytorch-onnx_rt22.1-py3.9      0b848dd4-e681-5599-be41-b5f6fccc6471  base
ai-function_0.1-py3.6          0cdb0f1e-5376-4f4d-92dd-da3b69aa9bda  base
shiny-r3.6                     0e6e79df-875e-4f24-8ae9-62dcc2148306  base
tensorflow_2.4-py3.7-horovod   1092590a-307d-563d-9b62-4eb7d64b3f22  base
pytorch_1.1-py3.6              10ac12d6-6b30-4ccd-8392-3e922c096a92  base
tensorflow_1.15-py3.6-ddl      111e41b3-de2d-5422-a4d6-bf776828c4b7  base
autoai-kb_rt22.2-py3.10        125b6d9

In [63]:
software_space_uid = client.software_specifications.get_uid_by_name("tensorflow_rt22.1-py3.9")

In [64]:
software_space_uid

'acd9c798-6974-5d2f-a657-ce06e986df4d'

In [65]:
 ls

 Dataset/   forest13.h5  'Forest fire'/   modell.h5   Train-model_new.tgz


In [66]:
model_details = client.repository.store_model(model= 'Train-model_new.tgz',
    meta_props={
        client.repository.ModelMetaNames.NAME:"CNN",
        client.repository.ModelMetaNames.TYPE:"tensorflow_2.7",
        client.repository.ModelMetaNames.SOFTWARE_SPEC_UID:software_space_uid}
    )


In [67]:
model_id = client.repository.get_model_id(model_details)

In [68]:
model_id

'bf4d141e-955d-46f1-a601-918d99da572a'

In [69]:
ls

 Dataset/   forest13.h5  'Forest fire'/   modell.h5   Train-model_new.tgz


In [70]:
import numpy as np
from tensorflow.keras.models import load_model
from tensorflow.keras.preprocessing import image

In [80]:
model=load_model('forest13.h5')

In [78]:
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3
import os, types
import pandas as pd
from botocore.client import Config
import ibm_boto3

def __iter__(self): return 0

# @hidden_cell
# The following code accesses a file in your IBM Cloud Object Storage. It includes your credentials.
# You might want to remove those credentials before you share the notebook.
cos_client = ibm_boto3.client(service_name='s3',
    ibm_api_key_id='bHfE0z3tYuwrhV-z0neNxGqprEFMVR3oO8JFhqdxMLD0',
    ibm_auth_endpoint="https://iam.cloud.ibm.com/oidc/token",
    config=Config(signature_version='oauth'),
    endpoint_url='https://s3.private.us.cloud-object-storage.appdomain.cloud')

bucket = 'forestfiredetection-donotdelete-pr-kwf7h4pt0wk0fy'
object_key = 'Forest fire.zip'

streaming_body_7 = cos_client.get_object(Bucket=bucket, Key=object_key)['Body']

# Your data file was loaded into a botocore.response.StreamingBody object.
# Please read the documentation of ibm_boto3 and pandas to learn more about the possibilities to load the data.
# ibm_boto3 documentation: https://ibm.github.io/ibm-cos-sdk-python/
# pandas documentation: http://pandas.pydata.org/
def __iter__(self): return 0